# ML for CyberSecurity

Name: Jash Rathod

Net ID: jsr10000

# Downloading Relevant Files

We will start by downloading the [CSAW-HackML-2020 GitHub Repo](https://github.com/csaw-hackml/CSAW-HackML-2020) and the [data files](https://drive.google.com/drive/folders/1Rs68uH8Xqa4j6UxG53wzD0uyI8347dSq?usp=sharing)

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 220 (delta 37), reused 17 (delta 17), pack-reused 178
Receiving objects: 100% (220/220), 83.78 MiB | 26.80 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [2]:
# All data: https://drive.google.com/drive/folders/1Rs68uH8Xqa4j6UxG53wzD0uyI8347dSq?usp=sharing

# bd_test.h5 - https://drive.google.com/file/d/1kxNACo0qFo8QdZgtGHvaA67p4h4RcNIy/view?usp=sharing
# bd_valid.h5 - https://drive.google.com/file/d/1DRKofqVdn2ioh44M45eYZHl_XAW9r3v4/view?usp=sharing
# test.h5 - https://drive.google.com/file/d/1HpahIi-RcvtaRoly_TbuoBzWUaAjVDgt/view?usp=sharing
# valid.h5 - https://drive.google.com/file/d/1nbB5tyUVClSaFvvg3hrFW4wOUj3GtNTf/view?usp=sharing

import gdown

!mkdir -p /content/CSAW-HackML-2020/lab3/data/bd /content/CSAW-HackML-2020/lab3/data/cl

gid_outpath_list = [("1kxNACo0qFo8QdZgtGHvaA67p4h4RcNIy", "/content/CSAW-HackML-2020/lab3/data/bd/bd_test.h5"),
                    ("1DRKofqVdn2ioh44M45eYZHl_XAW9r3v4", "/content/CSAW-HackML-2020/lab3/data/bd/bd_valid.h5"),
                    ("1HpahIi-RcvtaRoly_TbuoBzWUaAjVDgt", "/content/CSAW-HackML-2020/lab3/data/cl/test.h5"),
                    ("1nbB5tyUVClSaFvvg3hrFW4wOUj3GtNTf", "/content/CSAW-HackML-2020/lab3/data/cl/valid.h5")]

for x in gid_outpath_list:
    gdown.download(f"https://drive.google.com/uc?id={x[0]}", x[1])

Downloading...
From: https://drive.google.com/uc?id=1kxNACo0qFo8QdZgtGHvaA67p4h4RcNIy
To: /content/CSAW-HackML-2020/lab3/data/bd/bd_test.h5
100%|██████████| 398M/398M [00:02<00:00, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DRKofqVdn2ioh44M45eYZHl_XAW9r3v4
To: /content/CSAW-HackML-2020/lab3/data/bd/bd_valid.h5
100%|██████████| 716M/716M [00:06<00:00, 119MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HpahIi-RcvtaRoly_TbuoBzWUaAjVDgt
To: /content/CSAW-HackML-2020/lab3/data/cl/test.h5
100%|██████████| 398M/398M [00:03<00:00, 124MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nbB5tyUVClSaFvvg3hrFW4wOUj3GtNTf
To: /content/CSAW-HackML-2020/lab3/data/cl/valid.h5
100%|██████████| 716M/716M [00:04<00:00, 157MB/s]


In [3]:
!du -sh /content/CSAW-HackML-2020/lab3/models/bd_net.h5
!du -sh /content/CSAW-HackML-2020/lab3/models/bd_weights.h5

7.0M	/content/CSAW-HackML-2020/lab3/models/bd_net.h5
2.4M	/content/CSAW-HackML-2020/lab3/models/bd_weights.h5


# Implementation

We will now perform the following steps:

1. Import libraries
2. Load the dataset
3. Define the model class
4. Define function for pruning defense


In [4]:
# Import necessary libraries
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [5]:
# File paths for the datasets and the model
neural_network_model_path = "/content/CSAW-HackML-2020/lab3/models/bd_net.h5"
neural_network_model_weights = "/content/CSAW-HackML-2020/lab3/models/bd_weights.h5"

poisoned_dataset_path = "/content/CSAW-HackML-2020/lab3/data/bd/bd_valid.h5"
poisoned_test_path = "/content/CSAW-HackML-2020/lab3/data/bd/bd_test.h5"

clean_dataset_path = "/content/CSAW-HackML-2020/lab3/data/cl/valid.h5"
test_dataset_path = "/content/CSAW-HackML-2020/lab3/data/cl/test.h5"

In [ ]:
# Function to load data from a given file path
def load_dataset(file_path):
    # Open the file and read data
    file_data = h5py.File(file_path, 'r')
    images, labels = np.array(file_data['data']), np.array(file_data['label'])

    # Adjust the dimensions of the images
    images = images.transpose((0, 2, 3, 1))
    return images, labels

# Loading different datasets
clean_images_valid, clean_labels_valid = load_dataset(clean_dataset_path)
clean_images_test, clean_labels_test = load_dataset(test_dataset_path)
poisoned_images_valid, poisoned_labels_valid = load_dataset(poisoned_dataset_path)

In [ ]:
# Function to extract intermediate layer output from a model
def extract_intermediate_layer(model, layer_index):
    # Create a model to output the intermediate layer
    return keras.models.Model(inputs=model.input, outputs=model.layers[layer_index].output)

In [ ]:
# Class definition for the modified model
# This model outputs a repaired model "G"
# `call` takes argument `input_data` (which is "Dvalid"​, a validation dataset of clean, labelled images)
class ModifiedModel(keras.Model):
    def __init__(self, OriginalModel, PrunedModel):
        super(ModifiedModel, self).__init__()
        self.OriginalModel = OriginalModel
        self.PrunedModel = PrunedModel

    def call(self, input_data):
        original_predictions = np.argmax(self.OriginalModel(input_data), axis=1)
        pruned_predictions = np.argmax(self.PrunedModel(input_data), axis=1)
        combined_result = np.array([original_predictions[i] if original_predictions[i] == pruned_predictions[i] else 1283 for i in range(original_predictions.shape[0])])
        result_matrix = np.zeros((original_predictions.shape[0], 1284))
        result_matrix[np.arange(combined_result.size), combined_result] = 1
        return result_matrix

We will start at layer `6` for pruning (`0`-indexed)

In [ ]:
# Function to apply pruning defense mechanism
def apply_pruning_defense(threshold):
    # Specify the layer index for pruning
    target_layer_index = 6
    OriginalModel = keras.models.load_model(neural_network_model_path) # Model "B"
    PrunedModel = keras.models.load_model(neural_network_model_path)

    # Calculate and display original accuracy
    clean_labels_predicted = np.argmax(OriginalModel.predict(clean_images_valid), axis=1)
    original_accuracy = np.mean(np.equal(clean_labels_predicted, clean_labels_valid)) * 100
    print("Original accuracy on clean data: ", original_accuracy)

    # Calculate and display Attack Success Rate (ASR) of the poisoned model
    poisoned_labels_predicted = np.argmax(OriginalModel.predict(poisoned_images_valid), axis=1)
    attack_success_rate = np.mean(np.equal(poisoned_labels_predicted, poisoned_labels_valid)) * 100
    print("Attack Success Rate (ASR) of BadNet: ", attack_success_rate)

    # Get activations from the intermediate layer
    intermediate_layer_output = extract_intermediate_layer(OriginalModel, target_layer_index)
    activations = intermediate_layer_output.predict(clean_images_valid)

    # Compute average activations and start pruning
    average_activations = activations.mean(axis=(0, 1, 2))
    activation_indices_sorted = np.argsort(average_activations)

    # Iteratively prune channels
    for channel in tqdm(activation_indices_sorted):
        # Access and zero-out the target channel in the last layer
        target_layer = PrunedModel.layers[target_layer_index - 1]
        K.set_value(target_layer.kernel[:, :, :, channel], np.zeros_like(target_layer.kernel[:, :, :, channel]))
        K.set_value(target_layer.bias[channel], np.zeros_like(target_layer.bias[channel]))

        # Recalculate accuracy after each pruning step
        # clean_labels_predicted = np.argmax(PrunedModel.predict(clean_images_valid), axis=1)
        # pruned_accuracy = np.mean(np.equal(clean_labels_predicted, clean_labels_valid)) * 100
        pruned_accuracy = np.mean(np.equal(np.argmax(PrunedModel.predict(clean_images_valid), axis=1), clean_labels_valid)) * 100
        print('\nVal accuracy (after pruning channel', channel, '):', pruned_accuracy)

        # Stop pruning if the accuracy drop exceeds the threshold
        if abs(original_accuracy - pruned_accuracy) > threshold:
            break

    # Create an instance of the ModifiedModel class
    refined_model = ModifiedModel(OriginalModel, PrunedModel)

    # Evaluate the refined model on clean test data
    # clean_test_predictions = refined_model(clean_images_test)
    clean_test_accuracy = np.mean(np.equal(np.argmax(refined_model(clean_images_test), axis=1), clean_labels_test)) * 100
    print("Clean test accuracy of Refined Model: ", clean_test_accuracy)

    # Evaluate the refined model on poisoned validation data
    # poisoned_validation_predictions = refined_model(poisoned_images_valid)
    poisoned_validation_accuracy = np.mean(np.equal(np.argmax(refined_model(poisoned_images_valid), axis=1), poisoned_labels_valid)) * 100
    print("ASR of Refined Model: ", poisoned_validation_accuracy) # Attack success rate

    return PrunedModel

## IMPORTANT NOTE

The next 3 code cells produce the repaired models for different threshold values (2%, 4%, 10%). As these demand large System RAM, not all 3 cells can be run in succession. Hence, run only 1 cell for which you need the results. Then restart the runtime, run all the `Implementation` cells and run the other cell (and then the third).

Below, I have produced results for the 3 repaired models, where each run shares the validation, test accuracies and attack success rates.

In [ ]:
# Prune the model with a threshold of 2 and save the repaired model
repaired_model_threshold_2 = apply_pruning_defense(2)
repaired_model_threshold_2.save("model_repaired_threshold_2.h5")
del repaired_model_threshold_2

361/361 [==============================] - 3s 3ms/step
Original accuracy on clean data:  98.64899974019225
361/361 [==============================] - 1s 2ms/step
Attack Success Rate (ASR) of BadNet:  100.0
361/361 [==============================] - 1s 2ms/step


  0%|          | 0/60 [00:00<?, ?it/s]

361/361 [==============================] - 1s 2ms/step


  2%|▏         | 1/60 [00:02<02:27,  2.51s/it]


Val accuracy (after pruning channel 0 ): 98.64899974019225
361/361 [==============================] - 1s 2ms/step


...


Val accuracy (after pruning channel 1 ): 97.50584567420108
361/361 [==============================] - 1s 3ms/step


 73%|███████▎  | 44/60 [01:48<00:39,  2.47s/it]


Val accuracy (after pruning channel 29 ): 95.75647354291158


Clean test accuracy of Refined Model:  95.74434918160561
ASR of Refined Model:  100.0


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Prune the model with a threshold of 4 and save the repaired model
repaired_model_threshold_4 = apply_pruning_defense(4)
repaired_model_threshold_4.save("model_repaired_threshold_4.h5")

361/361 [==============================] - 4s 3ms/step
Original accuracy on clean data:  98.64899974019225
361/361 [==============================] - 1s 2ms/step
Attack Success Rate (ASR) of BadNet:  100.0
361/361 [==============================] - 1s 2ms/step


  0%|          | 0/60 [00:00<?, ?it/s]

361/361 [==============================] - 1s 2ms/step


  2%|▏         | 1/60 [00:02<02:24,  2.45s/it]


Val accuracy (after pruning channel 0 ): 98.64899974019225
361/361 [==============================] - 1s 2ms/step


...


Val accuracy (after pruning channel 56 ): 94.7172425738287
361/361 [==============================] - 1s 2ms/step


 78%|███████▊  | 47/60 [02:01<00:33,  2.58s/it]


Val accuracy (after pruning channel 46 ): 92.09318437689443


Clean test accuracy of Refined Model:  92.1278254091972
ASR of Refined Model:  99.9913397419243


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Prune the model with a threshold of 10 and save the repaired model
repaired_model_threshold_10 = apply_pruning_defense(10)
repaired_model_threshold_10.save("model_repaired_threshold_10.h5")

361/361 [==============================] - 9s 3ms/step
Original accuracy on clean data:  98.64899974019225
361/361 [==============================] - 1s 2ms/step
Attack Success Rate (ASR) of BadNet:  100.0
361/361 [==============================] - 1s 2ms/step


  0%|          | 0/60 [00:00<?, ?it/s]

361/361 [==============================] - 1s 3ms/step


  2%|▏         | 1/60 [00:02<02:54,  2.95s/it]


Val accuracy (after pruning channel 0 ): 98.64899974019225
361/361 [==============================] - 1s 2ms/step


...


Val accuracy (after pruning channel 11 ): 89.17467740538669
361/361 [==============================] - 1s 2ms/step


 85%|████████▌ | 51/60 [02:12<00:23,  2.60s/it]


Val accuracy (after pruning channel 54 ): 84.43751623798389


Clean test accuracy of Refined Model:  84.3335931410756
ASR of Refined Model:  77.015675067117


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Compiling all the results

In [ ]:
results_df_1 = pd.DataFrame({
                    "Threshold": [2, 4, 10],
                    "Clean test accuracy of Refined Model": [95.74434918160561, 92.1278254091972, 84.3335931410756],
                    "Attack Success Rate": [100.0, 99.9913397419243, 77.015675067117]
                })

results_df_1

,Threshold,Clean test accuracy of Refined Model,Attack Success Rate
0,2,95.744349,100.000000
1,4,92.127825,99.991340
2,10,84.333593,77.015675


### Clearly we observe and improvement in attack success rates, which indicates that as the threshold increases, the model develops better resistance to attacks.

# Obtaining the accuracy on clean test data and the attack success rate (on backdoored test data) as a function of the fraction of channels pruned (X)

In [ ]:
# Function to apply pruning defense and generate statistics
def apply_pruning_and_generate_stats():
    target_layer_index = 6
    OriginalModel = keras.models.load_model(neural_network_model_path)
    PrunedModel = keras.models.load_model(neural_network_model_path)

    # Evaluate the original model on clean and poisoned data
    original_accuracy = np.mean(np.equal(np.argmax(OriginalModel.predict(clean_images_test), axis=1), clean_labels_test)) * 100
    print("Original accuracy on clean data: ", original_accuracy)
    original_asr = np.mean(np.equal(np.argmax(OriginalModel.predict(poisoned_images_valid), axis=1), poisoned_labels_valid)) * 100
    print("\nAttack Success Rate (ASR) on poisoned data: ", original_asr)

    # Get intermediate layer activations
    intermediate_activations = extract_intermediate_layer(OriginalModel, target_layer_index).predict(clean_images_valid)
    average_activations = intermediate_activations.mean(axis=(0,1,2))
    sorted_activation_indices = np.argsort(average_activations)
    total_len = len(sorted_activation_indices)

    # Lists to store results for plotting
    clean_data_accuracies = []
    poisoned_data_asrs = []
    pruning_progresses = []

    # Apply pruning iteratively
    pruning_progress = 0
    for channel in sorted_activation_indices:
        # Zeroing out the weights of the target channel
        layer_to_prune = PrunedModel.layers[target_layer_index - 1]
        K.set_value(layer_to_prune.kernel[:, :, :, channel], np.zeros_like(layer_to_prune.kernel[:, :, :, channel]))
        K.set_value(layer_to_prune.bias[channel], np.zeros_like(layer_to_prune.bias[channel]))

        # Create a refined model instance and evaluate
        refined_model = ModifiedModel(OriginalModel, PrunedModel)
        # pruning_progress = (pruning_progress + 1) / len(sorted_activation_indices)
        print(f"\nPruning progress percentage: {(pruning_progress / total_len):.2f}")

        clean_data_accuracy = np.mean(np.equal(np.argmax(refined_model(clean_images_test), axis=1), clean_labels_test)) * 100
        print("Clean data accuracy of Modified Model: ", clean_data_accuracy)

        poisoned_data_asr = np.mean(np.equal(np.argmax(refined_model(poisoned_images_valid), axis=1), poisoned_labels_valid)) * 100
        print("ASR of Refined Model on poisoned data: ", poisoned_data_asr)

        # Store results for plotting
        clean_data_accuracies.append(clean_data_accuracy)
        poisoned_data_asrs.append(poisoned_data_asr)
        pruning_progresses.append(pruning_progress / total_len)
        pruning_progress = pruning_progress + 1

    return pruning_progresses, clean_data_accuracies, poisoned_data_asrs


In [ ]:
# Execute the pruning function and store the results
pruning_progresses, clean_data_accuracies, poisoned_data_asrs = apply_pruning_and_generate_stats()

361/361 [==============================] - 1s 2ms/step
Original accuracy on clean data:  98.64899974019225
361/361 [==============================] - 1s 2ms/step

Attack Success Rate (ASR) on poisoned data:  100.0
361/361 [==============================] - 1s 2ms/step

Pruning progress percentage: 0.00
Clean data accuracy of Modified Model:  98.64899974019225
ASR of Refined Model on poisoned data:  100.0

Pruning progress percentage: 0.02
Clean data accuracy of Modified Model:  98.64899974019225
ASR of Refined Model on poisoned data:  100.0

Pruning progress percentage: 0.03
Clean data accuracy of Modified Model:  98.64899974019225
ASR of Refined Model on poisoned data:  100.0

Pruning progress percentage: 0.05
Clean data accuracy of Modified Model:  98.64899974019225
ASR of Refined Model on poisoned data:  100.0

Pruning progress percentage: 0.07
Clean data accuracy of Modified Model:  98.64899974019225
ASR of Refined Model on poisoned data:  100.0

Pruning progress percentage: 0.08
C

In [ ]:
results_df_2 = pd.DataFrame({
                    "Pruning progress percentage": pruning_progresses,
                    "Clean test accuracy of Modified Model": clean_data_accuracies,
                    "Attack Success Rate (ASR)": poisoned_data_asrs
                })

results_df_2.to_csv("results.csv", index=False)

results_df_2

,Pruning progress percentage,Clean test accuracy of Modified Model,Attack Success Rate (ASR)
0,0.000000,98.649000,100.000000
1,0.016667,98.649000,100.000000
2,0.033333,98.649000,100.000000
3,0.050000,98.649000,100.000000
4,0.066667,98.649000,100.000000
5,0.083333,98.649000,100.000000
6,0.100000,98.649000,100.000000
7,0.116667,98.649000,100.000000
8,0.133333,98.649000,100.000000
9,0.150000,98.649000,100.000000


# Evaluating the Goodnet G models (`ModifiedModel`)

Below are the results for all the 3 goodnet models on the cleaned and poisoned test set.

In [6]:
!python3 CSAW-HackML-2020/lab3/eval.py CSAW-HackML-2020/lab3/data/cl/test.h5 CSAW-HackML-2020/lab3/data/bd/bd_test.h5 model_repaired_threshold_2.h5

2023-12-04 20:50:36.952484: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 20:50:36.952546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 20:50:36.952591: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 20:50:38.552445: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
401/401 [==============================] - 11s 28ms/step
Clean Classification accuracy: 95.90023382696803
401/401 [==============================] - 10s 25ms/step
Attack Success Rate: 100.0


In [7]:
!python3 CSAW-HackML-2020/lab3/eval.py CSAW-HackML-2020/lab3/data/cl/test.h5 CSAW-HackML-2020/lab3/data/bd/bd_test.h5 model_repaired_threshold_4.h5

2023-12-04 20:51:35.898735: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 20:51:35.898818: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 20:51:35.898866: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 20:51:37.251276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
401/401 [==============================] - 13s 31ms/step
Clean Classification accuracy: 92.29150428682775
401/401 [==============================] - 9s 23ms/step
Attack Success Rate: 99.98441153546376


In [8]:
!python3 CSAW-HackML-2020/lab3/eval.py CSAW-HackML-2020/lab3/data/cl/test.h5 CSAW-HackML-2020/lab3/data/bd/bd_test.h5 model_repaired_threshold_10.h5

2023-12-04 20:52:14.858570: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 20:52:14.859074: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 20:52:14.859148: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 20:52:16.190355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
401/401 [==============================] - 11s 28ms/step
Clean Classification accuracy: 84.54403741231489
401/401 [==============================] - 10s 25ms/step
Attack Success Rate: 77.20966484801247


| Model Name                     | Clean Classification Accuracy (%) | Attack Success Rate (%) |
|--------------------------------|-----------------------------------|-------------------------|
| model_repaired_threshold_2.h5  | 95.90                             | 100.00                  |
| model_repaired_threshold_4.h5  | 92.29                             | 99.98                   |
| model_repaired_threshold_10.h5 | 84.54                             | 77.21                   |
